In [2]:
pip install mlxtend

  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/73/da/d5d77a9a7a135c948dbf8d3b873655b105a152d69e590150c83d23c3d070/mlxtend-0.23.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.0/1.4 MB 487.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/1.4 MB 655.4 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.4 MB 774.0 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.4 MB 888.4 kB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.4 MB 1.0 MB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.4 MB 1.0 MB/s eta 0:00:02
   ----------- ---------------------------- 0.4/1.4 MB 1.1 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.4 MB 1.2 MB/s eta 0:00:01
   --------------- --------------------

In [6]:
import mlxtend

In [6]:
import pandas as pd
from os import chdir
#chdir("")
dataset = pd.read_csv('Jeux\market_basket.csv', header=0)
#vérifications
print(dataset.shape) #(1360, 303)
print(dataset.columns)
# transformation en valeurs booléennes (étape non obligatoire mais
# recommandée par MLxtend)
for col in dataset.columns:
    dataset[col] = dataset[col].map({1 : True, 0 : False})
    dataset[col].astype("boolean")


(1360, 303)
Index(['100_Watt_Lightbulb', '2pct_Milk', '40_Watt_Lightbulb',
       '60_Watt_Lightbulb', '75_Watt_Lightbulb', '98pct_Fat_Free_Hamburger',
       'AA_Cell_Batteries', 'Apple_Cinnamon_Waffles', 'Apple_Drink',
       'Apple_Fruit_Roll',
       ...
       'White_Bread', 'White_Wine', 'White_Zinfandel_Wine', 'Whole_Corn',
       'Whole_Green_Beans', 'Whole_Milk', 'Window_Cleaner', 'Wood_Polish',
       'flav_Fruit_Bars', 'flav_Ice'],
      dtype='object', length=303)


In [7]:
from mlxtend.frequent_patterns import apriori
taillemax = 4
freq_itemsets = apriori(dataset, min_support=0.025, max_len=taillemax,
use_colnames=True, verbose=0)
#type du résultat
type(freq_itemsets)
#nombre de motifs obtenus
print(freq_itemsets.shape) # (603, 2) ; 603 motifs fréquents
#liste des colonnes
print(freq_itemsets.columns)
#affichage des 15 premiers motifs
print(freq_itemsets.head(15))
#type de la colonne ’itemsets’
print(type(freq_itemsets.itemsets))
#affichage du premier élément
print(freq_itemsets.itemsets[0])

(603, 2)
Index(['support', 'itemsets'], dtype='object')
     support                    itemsets
0   0.030147        (100_Watt_Lightbulb)
1   0.109559                 (2pct_Milk)
2   0.037500         (60_Watt_Lightbulb)
3   0.031618         (75_Watt_Lightbulb)
4   0.093382  (98pct_Fat_Free_Hamburger)
5   0.031618         (AA_Cell_Batteries)
6   0.025735    (Apple_Cinnamon_Waffles)
7   0.026471               (Apple_Drink)
8   0.031618          (Apple_Fruit_Roll)
9   0.032353                 (Apple_Jam)
10  0.033088               (Apple_Jelly)
11  0.032353               (Apple_Sauce)
12  0.053676                    (Apples)
13  0.066912                   (Aspirin)
14  0.027941               (Avocado_Dip)
<class 'pandas.core.series.Series'>
frozenset({'100_Watt_Lightbulb'})


In [10]:
from mlxtend.frequent_patterns import association_rules
arules = association_rules(freq_itemsets, metric="confidence",
min_threshold=0.7)
#type du résultat
print(type(arules))
#nombre de règles générées
print(arules.shape) # (50, 9)
#liste des colonnes
print(arules.columns)
#affichage des 5 premières règles du résultat
print(arules.iloc[:5,:])


<class 'pandas.core.frame.DataFrame'>
(105, 10)
Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction', 'zhangs_metric'],
      dtype='object')
                             antecedents     consequents  antecedent support  \
0                         (Hot_Dog_Buns)      (Hot_Dogs)            0.058824   
1  (98pct_Fat_Free_Hamburger, 2pct_Milk)          (Eggs)            0.038235   
2  (98pct_Fat_Free_Hamburger, 2pct_Milk)  (Potato_Chips)            0.038235   
3                   (Aspirin, 2pct_Milk)          (Eggs)            0.034559   
4                   (Aspirin, 2pct_Milk)  (Potato_Chips)            0.034559   

   consequent support   support  confidence      lift  leverage  conviction  \
0            0.092647  0.041912    0.712500  7.690476  0.036462    3.156010   
1            0.122794  0.027206    0.711538  5.794565  0.022511    3.040980   
2            0.097794  0.027206    0.711

In [11]:
my_rules = arules.loc[:, ['antecedents','consequents', 'lift', 'conviction']]
print(my_rules.shape)
#pour un affichage plus lisible
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 10)
pd.set_option('display.precision', 3)
# sélection des colonnes à afficher
my_rules = arules.loc[:,['antecedents', 'consequents', 'lift', 'conviction']]
print(my_rules.shape)
#affichage des 5 premières règles
print(my_rules[:10])
#affichage des règles avec un lift supérieur ou égal à 7
print(my_rules[my_rules['lift'].ge(7.0)])
#trier les règles selon le lift (ordre décroissant) et afficher les 10
#meilleures règles
print(my_rules.sort_values(by='lift', ascending=False)[:10])
#affichage des règles contenant ’Eggs’ dans le conséquent
print(my_rules[my_rules['consequents'].ge({'Eggs'})])
#affichage des règles où le conséquent est exactement ’Eggs’
print(my_rules[my_rules['consequents'].eq({'Eggs'})])
#idem mais en triant selon le lift (ordre décroissant)
print(my_rules[my_rules['consequents'].eq({'Eggs'})].sort_values(by='lift', ascending=False))
#affichage des règles avec ’2pct_Milk’ contenu dans l’antécédent et ’
#Eggs' correspondant au conséquent
print(my_rules[my_rules['antecedents'].ge({'2pct_Milk'}) & my_rules['consequents'].eq({'Eggs'})].sort_values(by='lift', ascending=False
))


(105, 4)
(105, 4)
                             antecedents     consequents   lift  conviction
0                         (Hot_Dog_Buns)      (Hot_Dogs)  7.690       3.156
1  (98pct_Fat_Free_Hamburger, 2pct_Milk)          (Eggs)  5.795       3.041
2  (98pct_Fat_Free_Hamburger, 2pct_Milk)  (Potato_Chips)  7.276       3.128
3                   (Aspirin, 2pct_Milk)          (Eggs)  6.064       3.436
4                   (Aspirin, 2pct_Milk)  (Potato_Chips)  7.397       3.262
5                   (Aspirin, 2pct_Milk)   (White_Bread)  6.609       4.140
6                 (Bananas, White_Bread)     (2pct_Milk)  7.261       4.353
7                   (Bananas, 2pct_Milk)   (White_Bread)  6.833       4.735
8                           (Cola, Eggs)     (2pct_Milk)  6.638       3.265
9                      (Cola, 2pct_Milk)          (Eggs)  5.923       3.216
                                 antecedents     consequents   lift  \
0                             (Hot_Dog_Buns)      (Hot_Dogs)  7.690   
2   

In [15]:
#Q5
from mlxtend.frequent_patterns import apriori, association_rules

# Set the minimum support to 0.015
min_support = 0.01

# Generate frequent itemsets
freq_itemsets = apriori(dataset, min_support=min_support, max_len=taillemax, use_colnames=True, verbose=0)

# Generate association rules with minconf=0.7
rules = association_rules(freq_itemsets, metric="confidence", min_threshold=0.6)

# Display the number of frequent itemsets and association rules
print("Number of frequent itemsets:", freq_itemsets.shape[0])
print("Number of association rules:", rules.shape[0])

# Display the first few association rules
print(rules.head())


Number of frequent itemsets: 34212
Number of association rules: 61916
            antecedents  consequents  antecedent support  ...  leverage  \
0  (100_Watt_Lightbulb)       (Eggs)               0.030  ...     0.015   
1   (AA_Cell_Batteries)  (2pct_Milk)               0.032  ...     0.019   
2    (Apple_Fruit_Roll)  (2pct_Milk)               0.032  ...     0.019   
3         (Apple_Jelly)  (2pct_Milk)               0.033  ...     0.019   
4         (Baked_Beans)  (2pct_Milk)               0.032  ...     0.017   

   conviction  zhangs_metric  
0       2.248          0.823  
1       2.945          0.870  
2       2.945          0.870  
3       2.862          0.870  
4       2.449          0.856  

[5 rows x 10 columns]


In [12]:
#Trouvez quelques règles qui vous semblent intéressantes.

from mlxtend.frequent_patterns import apriori, association_rules

# Set the minimum support to 0.015 and other parameters
taillemax = 4
min_support = 0.015
min_confidence = 0.7

# Generate frequent itemsets
freq_itemsets = apriori(dataset, min_support=min_support, max_len=taillemax, use_colnames=True, verbose=0)

# Generate association rules with minconf=0.7
rules = association_rules(freq_itemsets, metric="confidence", min_threshold=min_confidence)

# Display the number of frequent itemsets and association rules
print("Number of frequent itemsets:", freq_itemsets.shape[0])
print("Number of association rules:", rules.shape[0])

# Display the first few association rules
print(rules.head())

# Filter rules based on desired criteria
interesting_rules = rules[(rules['support'] > min_support) & (rules['confidence'] > min_confidence)]

# Display the interesting rules
print("Number of interesting rules:", interesting_rules.shape[0])
print(interesting_rules)


Number of frequent itemsets: 4759
Number of association rules: 3265
                 antecedents  consequents  antecedent support  ...  leverage  \
0               (Buttermilk)  (2pct_Milk)               0.025  ...     0.015   
1     (Dishwasher_Detergent)  (2pct_Milk)               0.029  ...     0.017   
2  (Mushroom_Pizza_-_Frozen)  (2pct_Milk)               0.025  ...     0.016   
3                   (Sponge)  (2pct_Milk)               0.027  ...     0.016   
4        (Whole_Green_Beans)  (2pct_Milk)               0.022  ...     0.013   

   conviction  zhangs_metric  
0       3.027          0.866  
1       2.968          0.869  
2       3.364          0.873  
3       2.995          0.868  
4       2.968          0.863  

[5 rows x 10 columns]
Number of interesting rules: 3130
                                   antecedents    consequents  \
0                                 (Buttermilk)    (2pct_Milk)   
2                    (Mushroom_Pizza_-_Frozen)    (2pct_Milk)   
3            

In [16]:
# Filtrer les règles pour exclure celles qui incluent "Milk", "Eggs", et "Cola"
excluded_items = {"2pct_Milk", "Eggs", "Cola"}
filtered_rules = my_rules[~my_rules["antecedents"].apply(lambda x: any(item in excluded_items for item in x)) &
                          ~my_rules["consequents"].apply(lambda x: any(item in excluded_items for item in x))]

# Afficher les règles filtrées
print(filtered_rules)


                                 antecedents                 consequents  \
0                             (Hot_Dog_Buns)                  (Hot_Dogs)   
43             (Hamburger_Buns, White_Bread)  (98pct_Fat_Free_Hamburger)   
44  (98pct_Fat_Free_Hamburger, Potato_Chips)               (White_Bread)   
45   (98pct_Fat_Free_Hamburger, Wheat_Bread)               (White_Bread)   
48                   (Aspirin, Potato_Chips)               (White_Bread)   
..                                       ...                         ...   
88                    (Potatoes, Toothpaste)               (White_Bread)   
89                (Toothpaste, Sweet_Relish)               (White_Bread)   
90                (Toothpaste, Toilet_Paper)               (White_Bread)   
91                    (Tomatoes, Toothpaste)               (White_Bread)   
92                 (Wheat_Bread, Toothpaste)               (White_Bread)   

     lift  conviction  
0   7.690       3.156  
43  8.202       3.874  
44  5.976      

In [13]:
# Items to be excluded
excluded_items = ["2pct_Milk", "Eggs", "Cola"]

# Filter transactions to exclude those containing specified items
filtered_dataset = dataset[~dataset[excluded_items].any(axis=1)]

# Print the shape of the filtered dataset for verification
print(filtered_dataset.shape)


(1079, 303)


In [14]:
pip install xlrd>=2.0.1


Note: you may need to restart the kernel to use updated packages.


In [17]:
# Filtrer les règles pour inclure uniquement celles associées à "White_Bread"
white_bread_associations = my_rules[my_rules["antecedents"].apply(lambda x: "White_Bread" in x) | my_rules["consequents"].apply(lambda x: "White_Bread" in x)]

# Afficher toutes les colonnes
pd.set_option('display.max_columns', None)
print(white_bread_associations)


                                 antecedents     consequents   lift  \
5                       (Aspirin, 2pct_Milk)   (White_Bread)  6.609   
6                     (Bananas, White_Bread)     (2pct_Milk)  7.261   
7                       (Bananas, 2pct_Milk)   (White_Bread)  6.833   
12                       (Cola, White_Bread)     (2pct_Milk)  6.804   
13                         (Cola, 2pct_Milk)   (White_Bread)  6.258   
..                                       ...             ...    ...   
100           (Eggs, White_Bread, 2pct_Milk)    (Toothpaste)  8.995   
101  (Toothpaste, White_Bread, Potato_Chips)     (2pct_Milk)  7.569   
102    (Toothpaste, 2pct_Milk, Potato_Chips)   (White_Bread)  7.319   
103     (Toothpaste, White_Bread, 2pct_Milk)  (Potato_Chips)  7.726   
104   (Potato_Chips, White_Bread, 2pct_Milk)    (Toothpaste)  9.514   

     conviction  
5         4.140  
6         4.353  
7         4.735  
12        3.498  
13        3.461  
..          ...  
100       3.222  
101

In [19]:
# Filtrer les règles pour inclure uniquement celles associées à "White_Bread"
white_bread_associations = my_rules[my_rules["antecedents"].apply(lambda x: "White_Bread" in x) | my_rules["consequents"].apply(lambda x: "White_Bread" in x)]

# Afficher toutes les colonnes et toutes les lignes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Imprimer le DataFrame des règles associées à "White_Bread"
print(white_bread_associations)
print(len(white_bread_associations))

                                  antecedents                 consequents  \
5                        (Aspirin, 2pct_Milk)               (White_Bread)   
6                      (Bananas, White_Bread)                 (2pct_Milk)   
7                        (Bananas, 2pct_Milk)               (White_Bread)   
12                        (Cola, White_Bread)                 (2pct_Milk)   
13                          (Cola, 2pct_Milk)               (White_Bread)   
25                   (White_Bread, 2pct_Milk)                      (Eggs)   
26                      (Hot_Dogs, 2pct_Milk)               (White_Bread)   
33                  (Potato_Chips, 2pct_Milk)               (White_Bread)   
35                      (Potatoes, 2pct_Milk)               (White_Bread)   
36                  (Toilet_Paper, 2pct_Milk)               (White_Bread)   
37                      (2pct_Milk, Tomatoes)               (White_Bread)   
39                    (Toothpaste, 2pct_Milk)               (White_Bread)   

# Exercice 2


## 1. Chargement des données



In [15]:
'''import pandas as pd
from os import chdir
#chdir("mettre le chemin vers votre dossier de travail")
df = pd.read_excel('Jeux\titanic3.xls')
#vérification des données chargées
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.head())
'''

import pandas as pd
from os import chdir

# Assuming 'Jeux' is a folder in the current directory
# If not, provide the complete path to the file
file_path = r'Jeux\titanic3.xls'

# Read the Excel file
df = pd.read_excel(file_path)
print(df.head(20))
# Verification of the loaded data
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.head())



    pclass  survived                                             name  ...  \
0        1         1                    Allen, Miss. Elisabeth Walton  ...   
1        1         1                   Allison, Master. Hudson Trevor  ...   
2        1         0                     Allison, Miss. Helen Loraine  ...   
3        1         0             Allison, Mr. Hudson Joshua Creighton  ...   
4        1         0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  ...   
..     ...       ...                                              ...  ...   
15       1         0                              Baumann, Mr. John D  ...   
16       1         0                         Baxter, Mr. Quigg Edmond  ...   
17       1         1  Baxter, Mrs. James (Helene DeLaudeniere Chaput)  ...   
18       1         1                            Bazzani, Miss. Albina  ...   
19       1         0                             Beattie, Mr. Thomson  ...   

   boat   body                        home.dest  
0     2    Na

## 2. Préparation des données


In [22]:
from mlxtend.preprocessing import TransactionEncoder


#on enlève les attributs non voulus
df1 = df.drop(columns = ['name', 'ticket', 'home.dest', 'cabin', 'boat', 'body', 'sibsp', 'parch', 'fare', 'embarked'])
print(df1.shape)
print(df1.columns)
print(df1.head())
#ignorer les exemples où l'âge est inconnu
print(df1['age'].isnull().sum())
df2 = df1.dropna()
print(df2.shape)
print(df2.head())
#reste-t-il des valeurs manquantes ?
print(df2.isnull().values.any())
#discrétisation de l'âge
data = df2['age']
agemin = data.min()
agemax = data.max()
datad = pd.cut(data, bins=[agemin, 18, agemax],
labels=['child', 'adult'], include_lowest = True)
#ici, au lieu d'indiquer le nombre d'intervalles (par exemple, "bins
#=2"), on donne une liste de valeurs. Afin d'inclure le minimum, on
#ajoute "include_lowest=True".

print(datad.value_counts())
df2.insert(4, "aged", datad, True)
print(df2.head())
df2 = df2.drop(columns = ['age'])
df2 = df2.rename(columns={'aged': 'age'})
print(df2.head())
#renommage de valeurs
df2['pclass'] = df2['pclass'].map({1:'class=1st', 2:'class=2nd', 3:'class=3rd'})
df2['survived'] = df2['survived'].map({1:'survived=yes', 0:'survived=no'})
df2['sex'] = df2['sex'].map({'female':'sex=female', 'male':'sex=male'})
df2['age'] = df2['age'].map({'adult':'age=adult', 'child':'age=child'})
#changement de types
print(df2.dtypes)
df2['pclass'] = df2['pclass'].astype('category')
df2['survived'] = df2['survived'].astype('category')
df2['sex'] = df2['sex'].astype('category')
print(df2.dtypes)
print(df2.head())
#(facultatif) enregistrement des données préparées dans un fichier
df2.to_csv('titanic.csv', header=True, index=False)
#les données sont presque prêtes, il faut maintenant les mettre dans
#le format accepté par MLxtend
dataset = df2.values
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
#print(dataset)

(1309, 4)
Index(['pclass', 'survived', 'sex', 'age'], dtype='object')
   pclass  survived     sex     age
0       1         1  female  29.000
1       1         1    male   0.917
2       1         0  female   2.000
3       1         0    male  30.000
4       1         0  female  25.000
263
(1046, 4)
   pclass  survived     sex     age
0       1         1  female  29.000
1       1         1    male   0.917
2       1         0  female   2.000
3       1         0    male  30.000
4       1         0  female  25.000
False
age
adult    853
child    193
Name: count, dtype: int64
   pclass  survived     sex     age   aged
0       1         1  female  29.000  adult
1       1         1    male   0.917  child
2       1         0  female   2.000  child
3       1         0    male  30.000  adult
4       1         0  female  25.000  adult
   pclass  survived     sex    age
0       1         1  female  adult
1       1         1    male  child
2       1         0  female  child
3       1         0    m

## 3. Découverte de règles d’association


In [23]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Assuming df2 is your preprocessed DataFrame

# Convert the DataFrame to the transaction format
dataset = df2.values
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
dataset_transformed = pd.DataFrame(te_ary, columns=te.columns_)

# Calculate frequent itemsets with minsup = 0.5
minsup_threshold = 0.5
frequent_itemsets = apriori(dataset_transformed, min_support=minsup_threshold, use_colnames=True)

# Generate association rules with minconf = 0.8
minconf_threshold = 0.8
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minconf_threshold)

# Display the rules
print("Association Rules:")
print(rules)


Association Rules:
     antecedents  consequents  antecedent support  consequent support  \
0     (sex=male)  (age=adult)               0.629               0.815   
1  (survived=no)   (sex=male)               0.592               0.629   

   support  confidence   lift  leverage  conviction  zhangs_metric  
0    0.533       0.847  1.038     0.020       1.202          0.099  
1    0.500       0.845  1.343     0.128       2.392          0.626  


In [29]:
# Exemple en ajustant minsup et minconf
# Note: Ceci est un exemple, ajustez les valeurs selon vos besoins spécifiques

# Ajustez minsup en fonction de votre critère pour considérer les enfants
minsup_threshold = 0.3  # Exemple: Si minsup n'est pas assez bas, les enfants ne seront pas considérés

# Ajustez minconf en fonction de votre critère pour caractériser les survivants
minconf_threshold = 0.7  # Exemple: Ajustez en fonction de vos besoins pour caractériser les survivants

# Calculez les itemsets fréquents
frequent_itemsets = apriori(dataset_transformed, min_support=minsup_threshold, use_colnames=True)

# Générez les règles d'association
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minconf_threshold)

# Affichez les règles
print("Association Rules:")
print(rules)


Association Rules:
                 antecedents            consequents  antecedent support  \
0                (class=3rd)            (age=adult)               0.479   
1                 (sex=male)            (age=adult)               0.629   
2              (survived=no)            (age=adult)               0.592   
3             (survived=yes)            (age=adult)               0.408   
4                (class=3rd)          (survived=no)               0.479   
5                 (sex=male)          (survived=no)               0.629   
6              (survived=no)             (sex=male)               0.592   
7    (sex=male, survived=no)            (age=adult)               0.500   
8      (sex=male, age=adult)          (survived=no)               0.533   
9   (survived=no, age=adult)             (sex=male)               0.498   
10             (survived=no)  (sex=male, age=adult)               0.592   

    consequent support  support  confidence   lift  leverage  conviction  \
0   